In [1]:
from ophyd.sim import SynSignal

In [2]:
sig1 = SynSignal(name="sig1")

In [3]:
sig1.value

In [4]:
sig1.put(12000)

In [5]:
sig1.read()

{'sig1': {'value': 12000, 'timestamp': 1541095849.7885957}}

In [6]:
import numpy as np

In [7]:
import bluesky.plan_stubs as bps
import time

class MySynSignal(SynSignal):
    
    def my_random(self):
        # print("*"*20, self.exposure_time)
        # yield from bps.sleep(self.exposure_time)
        # time.sleep(self.exposure_time)
        return np.random.rand()

sig1 = MySynSignal(name="sig1")
sig1._func = sig1.my_random
sig1.exposure_time = 1

In [8]:
sig1.read()

{'sig1': {'value': None, 'timestamp': 1541095850.0154335}}

In [9]:
sig1.trigger()

DeviceStatus(device=sig1, done=False, success=False)

In [10]:
sig1.read()

{'sig1': {'value': None, 'timestamp': 1541095850.0154335}}

In [11]:
sig1.trigger()
sig1.read()

{'sig1': {'value': None, 'timestamp': 1541095850.0154335}}

In [12]:
from bluesky import RunEngine
import bluesky.plans as bp
from bluesky.simulators import summarize_plan

RE = RunEngine({})

In [13]:
for item in bp.count([sig1], num=3, delay=1.5):
    print(item)

stage: (MySynSignal(name='sig1', timestamp=1541095850.0154335)), (), {}
open_run: (None), (), {'detectors': ['sig1'], 'num_points': 3, 'num_intervals': 2, 'plan_args': {'detectors': ["MySynSignal(name='sig1', timestamp=1541095850.0154335)"], 'num': 3}, 'plan_name': 'count', 'hints': {'dimensions': [(('time',), 'primary')]}}
checkpoint: (None), (), {}
trigger: (MySynSignal(name='sig1', timestamp=1541095850.0154335)), (), {'group': 'trigger-83d787'}
wait: (None), (), {'group': 'trigger-83d787'}
create: (None), (), {'name': 'primary'}
read: (MySynSignal(name='sig1', timestamp=1541095850.0154335)), (), {}
save: (None), (), {}
sleep: (None), (1.4994683265686035,), {}
checkpoint: (None), (), {}
trigger: (MySynSignal(name='sig1', timestamp=1541095850.0154335)), (), {'group': 'trigger-658647'}
wait: (None), (), {'group': 'trigger-658647'}
create: (None), (), {'name': 'primary'}
read: (MySynSignal(name='sig1', timestamp=1541095850.0154335)), (), {}
save: (None), (), {}
sleep: (None), (1.4997920

In [14]:
RE(bp.count([sig1], num=3, delay=2, md=dict(purpose="demo today for Chen", my_cat="sleeping", a_bool=True)), print)

start {'uid': '64a03bcc-9709-472d-8702-8055a5ff398c', 'time': 1541095850.1322348, 'scan_id': 1, 'purpose': 'demo today for Chen', 'num_intervals': 2, 'num_points': 3, 'a_bool': True, 'plan_args': {'detectors': ["MySynSignal(name='sig1', timestamp=1541095850.0154335)"], 'num': 3}, 'detectors': ['sig1'], 'plan_name': 'count', 'hints': {'dimensions': [(('time',), 'primary')]}, 'my_cat': 'sleeping', 'plan_type': 'generator'}
descriptor {'run_start': '64a03bcc-9709-472d-8702-8055a5ff398c', 'time': 1541095851.1385465, 'data_keys': {'sig1': {'source': 'SIM:sig1', 'dtype': 'number', 'shape': [], 'precision': 3, 'object_name': 'sig1'}}, 'uid': '013a9ab5-2c84-4a2c-b9d5-dcee1c8cdf81', 'configuration': {'sig1': {'data': {'sig1': 0.03806177915207132}, 'timestamps': {'sig1': 1541095851.1377492}, 'data_keys': {'sig1': {'source': 'SIM:sig1', 'dtype': 'number', 'shape': [], 'precision': 3}}}}, 'name': 'primary', 'hints': {'sig1': {'fields': []}}, 'object_keys': {'sig1': ['sig1']}}
event {'descriptor': 

('64a03bcc-9709-472d-8702-8055a5ff398c',)

In [15]:
def my_count(count_time=1, delay=0):
    sig1.exposure_time = count_time
    yield from bp.count([sig1], num=3, delay=delay, md=dict(purpose="demo today for Chen", my_cat="eating", a_bool=True))

In [16]:
RE(my_count(1), print)

start {'uid': '47ceaebc-3c9a-4944-9df8-d5309a0e2f70', 'time': 1541095856.1929536, 'scan_id': 2, 'purpose': 'demo today for Chen', 'num_intervals': 2, 'num_points': 3, 'a_bool': True, 'plan_args': {'detectors': ["MySynSignal(name='sig1', value=0.2791873123916916, timestamp=1541095855.140615)"], 'num': 3}, 'detectors': ['sig1'], 'plan_name': 'count', 'hints': {'dimensions': [(('time',), 'primary')]}, 'my_cat': 'eating', 'plan_type': 'generator'}
descriptor {'run_start': '47ceaebc-3c9a-4944-9df8-d5309a0e2f70', 'time': 1541095857.1981814, 'data_keys': {'sig1': {'source': 'SIM:sig1', 'dtype': 'number', 'shape': [], 'precision': 3, 'object_name': 'sig1'}}, 'uid': '4bf2681a-d344-4fd6-a94a-4c351a3d7ae7', 'configuration': {'sig1': {'data': {'sig1': 0.007962871548252037}, 'timestamps': {'sig1': 1541095857.196816}, 'data_keys': {'sig1': {'source': 'SIM:sig1', 'dtype': 'number', 'shape': [], 'precision': 3}}}}, 'name': 'primary', 'hints': {'sig1': {'fields': []}}, 'object_keys': {'sig1': ['sig1']}

('47ceaebc-3c9a-4944-9df8-d5309a0e2f70',)

In [17]:
sig1.trigger()
sig1.read()

{'sig1': {'value': 0.21343721371080848, 'timestamp': 1541095859.2084908}}

In [18]:
from bluesky.callbacks.best_effort import BestEffortCallback
bec = BestEffortCallback()
RE.subscribe(bec)


2

In [19]:
RE(my_count(1, 2))

Transient Scan ID: 3     Time: 2018/11/01 13:10:59
Persistent Unique Scan ID: 'a09ee9c1-c630-4557-b75c-88533eb8cfad'
New stream: 'primary'
+-----------+------------+
|   seq_num |       time |
+-----------+------------+
|         1 | 13:11:00.4 |
|         2 | 13:11:02.4 |
|         3 | 13:11:04.4 |
+-----------+------------+
generator count ['a09ee9c1'] (scan num: 3)





('a09ee9c1-c630-4557-b75c-88533eb8cfad',)